In [1]:
import numpy as np
import pickle
import pandas as pd
from tqdm import tqdm
from math import log
from scipy.sparse import coo_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats import mode
from experiment import EarlyStop, train_model
import os
import sys
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.multiprocessing as multiprocessing

import utils.Constant as CONSTANT
from dataloader import UIRTDatset
from evaluation import Evaluator

import warnings

warnings.filterwarnings("ignore")

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

In [2]:
gpu = 0
gpu = str(gpu)
os.environ["CUDA_VISIBLE_DEVICES"] = gpu 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_dir = "data/recsys_data"
dataset = "ml-1m"
min_user_per_item = 1
min_item_per_user = 10

# leave-k-out
leave_k = 5
popularity_order = True

dataset = UIRTDatset(data_dir, dataset, min_user_per_item, min_item_per_user, leave_k, popularity_order)

data loaded!


In [3]:
num_users, num_items = dataset.num_users, dataset.num_items
# mainstream score, jaccard similarity
train_mat = dataset.train_matrix.tocoo().toarray()
user_pop = np.sum(train_mat, axis=1)
Jaccard_mat = np.matmul(train_mat, train_mat.T)
deno = user_pop.reshape((-1, 1)) + user_pop.reshape((1, -1)) - Jaccard_mat + 1e-7
Jaccard_mat /= deno
Jaccard_mat = Jaccard_mat + np.eye(num_users) * -9999
Jaccard_mat = Jaccard_mat[np.where(Jaccard_mat > -1)].reshape((num_users, num_users - 1))
MS_similarity = np.mean(Jaccard_mat, axis=1)
similarity_dir = os.path.join(dataset.data_dir, dataset.data_name, 'mainstream_scores')
if not os.path.exists(similarity_dir):
    os.mkdir(similarity_dir)
similarity_file = os.path.join(similarity_dir, 'MS_similarity.npy')
with open(similarity_file, "wb") as f:
    np.save(f, MS_similarity)

In [4]:
# check = np.load('./data/recsys_data/ml-1m/mainstream_scores/gate_initialize.npy')
# print(check)